In [1]:
import os                                   # Get NC Files from Directory
from netCDF4 import Dataset                 # Read / Write NetCDF4 files
import matplotlib.pyplot as plt             # Plotting library
import cartopy, cartopy.crs as ccrs         # Plot maps
import cartopy.io.shapereader as shpreader  # Import shapefiles
import numpy as np                          # Import the Numpy package
import math                                 # Mathematical functions
from datetime import datetime               # Basic Dates and time types
from datetime import timedelta              # Substract hours from datetime

In [2]:
# Infos da Regiao

# Select Files
dt = 'OR_ABI-L2-TPWF-M6_G16_s2019'

# Path to save
mypath= 'C:\\Users\\Durso\\Documentos\\_Iniciacao Cientifica\\NC Files\\_Imagens\\Rio de Janeiro\\'

# Area of Plot
extent = [-44.64728755494618, -23.380671724510133, -41.74140391311923, -21.60441389915291] # Min lon, Max lon, Min lat, Max lat

In [3]:
# Path File
path = 'C:\\Users\\Durso\\Documentos\\_Iniciacao Cientifica\\NC Files\\Total Precipitable Water\\'

# Get Files
oldfl = os.listdir(path)

# Specific Files
fl = [x for x in oldfl if x.startswith(dt)]

In [4]:
# for NetCDF4 Dataset!

# Functions to convert lat / lon extent to array indices 
def geo2grid(lat, lon, nc):

    # Apply scale and offset 
    xscale, xoffset = nc.variables['x'].scale_factor, nc.variables['x'].add_offset
    yscale, yoffset = nc.variables['y'].scale_factor, nc.variables['y'].add_offset
    
    x, y = latlon2xy(lat, lon)
    col = (x - xoffset)/xscale
    lin = (y - yoffset)/yscale
    return int(lin), int(col)

def latlon2xy(lat, lon):
    # goes_imagery_projection:semi_major_axis
    req = 6378137 # meters
    #  goes_imagery_projection:inverse_flattening
    invf = 298.257222096
    # goes_imagery_projection:semi_minor_axis
    rpol = 6356752.31414 # meters
    e = 0.0818191910435
    # goes_imagery_projection:perspective_point_height + goes_imagery_projection:semi_major_axis
    H = 42164160 # meters
    # goes_imagery_projection: longitude_of_projection_origin
    lambda0 = -1.308996939

    # Convert to radians
    latRad = lat * (math.pi/180)
    lonRad = lon * (math.pi/180)

    # (1) geocentric latitude
    Phi_c = math.atan(((rpol * rpol)/(req * req)) * math.tan(latRad))
    # (2) geocentric distance to the point on the ellipsoid
    rc = rpol/(math.sqrt(1 - ((e * e) * (math.cos(Phi_c) * math.cos(Phi_c)))))
    # (3) sx
    sx = H - (rc * math.cos(Phi_c) * math.cos(lonRad - lambda0))
    # (4) sy
    sy = -rc * math.cos(Phi_c) * math.sin(lonRad - lambda0)
    # (5)
    sz = rc * math.sin(Phi_c)

    # x,y
    x = math.asin((-sy)/math.sqrt((sx*sx) + (sy*sy) + (sz*sz)))
    y = math.atan(sz/sx)

    return x, y

# Function to convert lat / lon extent to GOES-16 extents
def convertExtent2GOESProjection(extent):
    # GOES-16 viewing point (satellite position) height above the earth
    GOES16_HEIGHT = 35786023.0
    # GOES-16 longitude position
    GOES16_LONGITUDE = -75.0
    
    a, b = latlon2xy(extent[1], extent[0])
    c, d = latlon2xy(extent[3], extent[2])
    return (a * GOES16_HEIGHT, c * GOES16_HEIGHT, b * GOES16_HEIGHT, d * GOES16_HEIGHT)


mypath = mypath + 'Total Precipitable Water\\'

if not os.path.exists(mypath):
    os.makedirs(mypath)

In [5]:
for i in fl:

    # Open the GOES-R image
    file = Dataset(path+i)

    # Convert lat/lon to grid-coordinates
    lly, llx = geo2grid(extent[1], extent[0], file)
    ury, urx = geo2grid(extent[3], extent[2], file)

    # Get the pixel values
    data = file.variables['TPW'][ury:lly, llx:urx] # Regiao Recortada

    # Compute data-extent in GOES projection-coordinates
    img_extent = convertExtent2GOESProjection(extent)

    # Choose the plot size (width x height, in inches)
    plt.figure(figsize=(10,10))

    # Use the Geostationary projection in cartopy
    ax = plt.axes(projection=ccrs.Geostationary(central_longitude=-75.0, satellite_height=35786023.0))

    # Add and Plot Shapefiles
    shapefile = list(shpreader.Reader('C:\\Users\\Durso\\Documentos\\_Iniciacao Cientifica\\Countries\\Brasil\\BR_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=1)
    
    #ax.add_geometries(shapefile2, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=1) # Pay attention to #comment or not

    # Set cmap
    cmap = plt.cm.get_cmap("jet").copy()
    
    # Plot the image
    img = plt.imshow(data, vmin = 0, vmax = 60, origin = 'upper', extent = img_extent, cmap = cmap)

    # Set the color limits
    img.cmap.set_under('purple')
    img.cmap.set_over('brown')
    img.set_clim(20,50)

    # Add borders and gridlines
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.8)
    ax.gridlines(color='black', alpha=0.5, linestyle='--', linewidth=0.5)

    # Add a colorbar
    plt.colorbar(img, label = 'Precipitable Water (mm)', extend = 'neither', orientation = 'horizontal', aspect = 30, pad = 0.05, fraction = 0.03)

    #-----------------------------------------------------------------------------------------------------------
    # Given timestamp in string
    time_str = file.__dict__['time_coverage_start']
    date_format_str = '%Y-%m-%dT%H:%M:%S.%fZ'

    # create datetime object from timestamp string
    given_time = datetime.strptime(time_str, date_format_str)

    # Subtract 2 hours from datetime object
    final_time = given_time - timedelta(hours=3)

    # Add a title
    plt.title('GOES-16 Total Precipitable Water', fontweight='bold', fontsize=15, loc='left')
    plt.title('Full Disk\n{}'.format(final_time.strftime('%d %B %Y %H:%M BRT')),
              loc='right')

    #-----------------------------------------------------------------------------------------------------------
    # Save the image
    plt.savefig(fname = mypath + 'G16_TPWF_PW_' + final_time.strftime('%d_%B_%Y_%H%MBRT') + '.jpg', bbox_inches='tight', pad_inches=0, dpi=300)

    plt.close()